UWAGI:
Organizacja powinna być inna - najpierw advanced numpy, potem Internal Organization... potem memory [strides, changing shape, np data structures]

In [1]:
import numpy as np

# Advanced NumPy

## broadcasting

The term **broadcasting** describes how NumPy treats arrays with different shapes during arithmetic operations. Subject to certain constraints, the smaller array is “broadcast” across the larger array so that they have compatible shapes.

NumPy operations are usually done on pairs of arrays on an element-by-element basis. In the simplest case, the two arrays must have exactly the same shape, as in the following example:

In [2]:
a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 2.0, 2.0])
a * b

array([2., 4., 6.])

NumPy’s broadcasting rule relaxes this constraint when the arrays’ shapes meet certain constraints. The simplest broadcasting example occurs when an array and a scalar value are combined in an operation:

In [3]:
a = np.array([1.0, 2.0, 3.0])
b = 2.0
a * b

array([2., 4., 6.])

The result matches the previous example where `b` was an array. Conceptually, `b` is stretched to match `a`'s shape, with copies of the scalar. However, NumPy optimizes this without creating actual copies, ensuring efficient memory and computation.

![Broadcasting](Images/broadcasting.png)

### General broadcasting rules

When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing (i.e. rightmost) dimension and works its way left. Two dimensions are compatible when

1. they are equal, or
2. one of them is 1.

Input arrays can have different dimensions. The result matches the array with the most dimensions, using the largest size per dimension, assuming missing dimensions are size one.

### Example 1

Scaling RGB values for an image

In [4]:
image = np.random.rand(256, 256, 3)  # 256x256 image with 3 color channels
scale = np.array([0.5, 1.0, 1.5])  # Different scaling factors for each channel

# Broadcasting operation
scaled_image = image * scale

print("Image shape:", image.shape)
print("Scale shape:", scale.shape)
print("Result shape:", scaled_image.shape)
print()

Image shape: (256, 256, 3)
Scale shape: (3,)
Result shape: (256, 256, 3)



### Example 2

Broadcasting with different shaped arrays

In [5]:
A = np.random.rand(8, 1, 6, 1)  # 4D array
B = np.random.rand(   7, 1, 5)  # 3D array

# Broadcasting operation
result = A * B

print("A shape:", A.shape)
print("B shape:", B.shape)
print("Result shape:", result.shape)

A shape: (8, 1, 6, 1)
B shape: (7, 1, 5)
Result shape: (8, 7, 6, 5)


### Example 3

Examples of Arrays that can`t be broadcast.

In [6]:
try:
    A = np.random.rand(3)
    B = np.random.rand(4)
    result = A + B 
except ValueError as e:
    print(e)

operands could not be broadcast together with shapes (3,) (4,) 


In [7]:
try:
    A = np.random.rand(   2, 1)
    B = np.random.rand(8, 4, 3)
    result = A + B 
except ValueError as e:
    print(e)

operands could not be broadcast together with shapes (2,1) (8,4,3) 


### Example 4

Broadcasting when a 1-d array is added to a 2-d array

In [8]:
A = np.array([[ 0.0,  0.0,  0.0],
              [10.0, 10.0, 10.0],
              [20.0, 20.0, 20.0],
              [30.0, 30.0, 30.0]])
B = np.array([1.0, 2.0, 3.0])
A + B

array([[ 1.,  2.,  3.],
       [11., 12., 13.],
       [21., 22., 23.],
       [31., 32., 33.]])

![Broadcastin1D2D](Images/broadcasting_2.png)

### Example 5

Broadcasting provides a convenient way of taking the outer product (or any other outer operation) of two arrays. The following example shows an outer addition operation of two 1-d arrays

In [9]:
a = np.array([0.0, 10.0, 20.0, 30.0])
b = np.array([1.0, 2.0, 3.0])
a[:, np.newaxis] + b

array([[ 1.,  2.,  3.],
       [11., 12., 13.],
       [21., 22., 23.],
       [31., 32., 33.]])

![Broadcasting](Images/broadcasting_4.png)

Here the newaxis index operator inserts a new axis into a, making it a two-dimensional 4x1 array. Combining the 4x1 array with b, which has shape (3,), yields a 4x3 array.

## structured arrays

Structured arrays are ndarrays whose datatype is a composition of simpler datatypes organized as a sequence of named fields. For example:

In [10]:
dt = np.dtype([('time', [('min', np.int64), ('sec', np.int64)]),
               ('temp', float)])
x = np.array([((10, 0), 2), ((10,30), 3), ((11,00), 5)], 
             dtype=dt)
x

array([((10,  0), 2.), ((10, 30), 3.), ((11,  0), 5.)],
      dtype=[('time', [('min', '<i8'), ('sec', '<i8')]), ('temp', '<f8')])

In [11]:
print(f'x[time]: {x["time"]}')

x[time]: [(10,  0) (10, 30) (11,  0)]


In [12]:
print(f'x[temp]: {x["temp"]}')

x[temp]: [2. 3. 5.]


### numpy.fromfile

Construct an array from data in a text or binary file.

In [13]:
import tempfile
fname = tempfile.mkstemp()[1]
x.tofile(fname)

In [14]:
np.fromfile(fname, dtype=dt)

array([((10,  0), 2.), ((10, 30), 3.), ((11,  0), 5.)],
      dtype=[('time', [('min', '<i8'), ('sec', '<i8')]), ('temp', '<f8')])

## copies and views

The main difference between a copy and a view of an array is that the copy is a new array, and the view is just a view of the original array.

The **copy** owns the data and any changes made to the copy will not affect original array, and any changes made to the original array will not affect the copy.

The **view** does not own the data and any changes made to the view will affect the original array, and any changes made to the original array will affect the view.

### np.view()

In [15]:
x = np.arange(10)
arr_view1 = x.view()
arr_view2 = x[1:5]

print(f'x: {x}')
print(f'x.view(): {arr_view1}')
print(f'x[1:3]: {arr_view2}')

x: [0 1 2 3 4 5 6 7 8 9]
x.view(): [0 1 2 3 4 5 6 7 8 9]
x[1:3]: [1 2 3 4]


In [16]:
x[3] = 10

print(f'x: {x}')
print(f'x.view(): {arr_view1}')
print(f'x[1:3]: {arr_view2}')

x: [ 0  1  2 10  4  5  6  7  8  9]
x.view(): [ 0  1  2 10  4  5  6  7  8  9]
x[1:3]: [ 1  2 10  4]


In [17]:
arr_view1[4] = 123

print(f'x: {x}')
print(f'x.view(): {arr_view1}')
print(f'x[1:3]: {arr_view2}')

x: [  0   1   2  10 123   5   6   7   8   9]
x.view(): [  0   1   2  10 123   5   6   7   8   9]
x[1:3]: [  1   2  10 123]


### np.copy()

In [18]:
x = np.arange(10)
arr_copy = x.copy()

print(f'x: {x}')
print(f'x.copy(): {arr_copy}')

x: [0 1 2 3 4 5 6 7 8 9]
x.copy(): [0 1 2 3 4 5 6 7 8 9]


In [19]:
x[3] = 10

print(f'x: {x}')
print(f'x.copy(): {arr_copy}')

x: [ 0  1  2 10  4  5  6  7  8  9]
x.copy(): [0 1 2 3 4 5 6 7 8 9]


### How to tell if the array is a view or a copy

The *base* attribute of the ndarray makes it easy to tell if an array is a view or a copy. The base attribute of a view returns the original array while it returns None for a copy.

In [20]:
print(f'x.base: {x.base}')
print(f'arr_view1.base: {arr_view1.base}')
print(f'arr_copy.base: {arr_copy.base}')

x.base: None
arr_view1.base: [  0   1   2  10 123   5   6   7   8   9]
arr_copy.base: None


### Copy library 

Lets compare np.copy() and np.view() with copy.deepcopy(), copy.copy().

Shallow copy creates a new object that references the same inner objects as the original, sharing data references. Changes to inner objects affect all references, but changes to top-level objects may not.

When you change the inner item (origin[1][0]), all collections reflect the change:

In [21]:
from copy import copy

# changing inner item
origin = [[1], [2], [3]]
shallow_copy = copy(origin)
referenced = origin
origin[1][0] = 0

print(origin)     
print(shallow_copy) 
print(referenced) 

[[1], [0], [3]]
[[1], [0], [3]]
[[1], [0], [3]]


When you change a top-level item (origin[1]), the reference reflects the change, but the shallow copy remains unchanged because it shares references to the inner lists:

In [22]:
# changing a top-level item
origin = [[1], [2], [3]]
shallow_copy = copy(origin)
referenced = origin
origin[1] = 0

print(origin)      
print(shallow_copy) 
print(referenced)  

[[1], 0, [3]]
[[1], [2], [3]]
[[1], 0, [3]]


Deep copy duplicates the entire structure, including all nested sublists, ensuring independence:

In [23]:
from copy import deepcopy

origin = [[1],[2],[3]]
deep_copy = deepcopy(origin)
referenced = origin
origin[1][0]= 0

print(origin) 
print(deep_copy) 
print(referenced) 

[[1], [0], [3]]
[[1], [2], [3]]
[[1], [0], [3]]


### Summary

Here’s the key difference between references, shallow copies, and deep copies:
- `copy.deepcopy()` → Creates a completely independent copy of an object, including all nested elements. Changes in the original object do not affect the deep copy.
- `References` → Simply point to the same object, meaning any modification to the original will be reflected in all references.
- `copy.copy()` → Creates a shallow copy, duplicating only the outer structure of an object, but the inner elements remain shared with the original.
- `np.copy()` → In NumPy, this creates a fully independent copy of an array, duplicating all data.
- `np.view()` → Creates a new view of the same data, meaning changes in one will affect the other.

## sub-classing ndarray

### Why Subclass `ndarray`?
Subclassing `ndarray` allows you to extend its functionality, making it suitable for specialized tasks, such as:
- **Adding Metadata:** Attach custom attributes, such as physical units (e.g., meters, seconds).
- **Custom Operations:** Override arithmetic operations or behaviors for specific scientific or domain-specific needs.
- **Efficient Data Handling:** Maintain memory efficiency by leveraging `ndarray`'s slicing and broadcasting capabilities while adding specialized behaviors.

### Key Ways to Create Subclassed Instances
1. **Explicit Constructor Call:** Create an instance directly using `MySubClass(params)`.
   ```python
   class MyArray(np.ndarray):
       pass
   arr = MyArray((3,))  # Explicit constructor
   ```

2. **View Casting:** Cast an existing `ndarray` to a subclass.
   ```python
   data = np.array([1, 2, 3])
   subclassed_view = data.view(MyArray)
   ```

3. **New from Template:** Create a new instance from a template, such as slicing or copying.
   ```python
   sliced = subclassed_view[:2]  # Slice from template
   ```

### Implications for Subclassing

To handle these three paths, NumPy requires you to manage two special methods:
1. `__new__`:
This is where object creation happens (instead of __init__, which is more common in Python). It initializes the basic array structure.
2. `__array_finalize__`:
This method is called after creating a view or new instance from a template. You use it to ensure attributes specific to your subclass are copied or set correctly.

### Example 1: `__new__` and `__init__`

`__new__` is a standard Python method, and, if present, is called before `__init__` when we create a class instance.

In [24]:
class C:
    def __new__(cls, *args):
        print('Cls in __new__:', cls)
        print('Args in __new__:', args)
        # The `object` type __new__ method takes a single argument.
        return object.__new__(cls)
    def __init__(self, *args):
        print('type(self) in __init__:', type(self))
        print('Args in __init__:', args)

In [25]:
c = C('hello')

Cls in __new__: <class '__main__.C'>
Args in __new__: ('hello',)
type(self) in __init__: <class '__main__.C'>
Args in __init__: ('hello',)


As you can see, the object can be initialized in the `__new__` method or the `__init__` method, or both, and in fact ndarray does not have an `__init__` method, because all the initialization is done in the `__new__`method.

Why use `__new__` rather than just the usual `__init__`? Because in some cases, as for ndarray, we want to be able to return an object of some other class. Consider the following:

In [26]:
class D(C):
    def __new__(cls, *args):
        print('D cls is:', cls)
        print('D args in __new__:', args)
        return C.__new__(C, *args)

    def __init__(self, *args):
        # we never get here
        print('In D __init__')

In [27]:
d = D('Hello')

D cls is: <class '__main__.D'>
D args in __new__: ('Hello',)
Cls in __new__: <class '__main__.C'>
Args in __new__: ('Hello',)


The definition of C is the same as before, but for D, the `__new__` method returns an instance of class C rather than D. Note that the `__init__` method of D does not get called. 

When taking a view of a subclassed ndarray, the subclass type must be preserved. NumPy solves this by calling something like:
```python
obj = np.ndarray.__new__(subtype, shape, ...)
```
Here, subtype is the subclass (like MyArray, C, D), ensuring that the resulting view belongs to the subclass instead of reverting to a plain ndarray.

But when `np.ndarray.__new__()` creates the subclass instance, it knows nothing about what custom logic you implemented in your subclass's `__new__` method. So any subclass-specific attributes or initializations may be missing in the returned view.

Why Not Call subdtype.`__new__()` Directly?

The reason is compatibility: subclasses might not define a `__new__` method with the exact same signature as `ndarray.__new__`.

### Example 2: `__array_finalize__`

`MySubClass.__new__` method only gets called in the case of the explicit constructor call, so we can’t rely on `MySubClass.__new__` or `MySubClass.__init__` to deal with the view casting and new-from-template. It turns out that `MySubClass.__array_finalize__` does get called for all three methods of object creation, so this is where our object creation housekeeping usually goes.

In [28]:
class InfoArray(np.ndarray):

    def __new__(subtype, shape, dtype=float, buffer=None, offset=0,
                strides=None, order=None, info=None):
        # Create the ndarray instance of our type, given the usual
        # ndarray input arguments.  This will call the standard
        # ndarray constructor, but return an object of our type.
        # It also triggers a call to InfoArray.__array_finalize__
        obj = super().__new__(subtype, shape, dtype,
                              buffer, offset, strides, order)
        # set the new 'info' attribute to the value passed
        obj.info = info
        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        # ``self`` is a new object resulting from
        # ndarray.__new__(InfoArray, ...), therefore it only has
        # attributes that the ndarray.__new__ constructor gave it -
        # i.e. those of a standard ndarray.
        #
        # We could have got to the ndarray.__new__ call in 3 ways:
        # From an explicit constructor - e.g. InfoArray():
        #    obj is None
        #    (we're in the middle of the InfoArray.__new__
        #    constructor, and self.info will be set when we return to
        #    InfoArray.__new__)
        if obj is None: return
        # From view casting - e.g arr.view(InfoArray):
        #    obj is arr
        #    (type(obj) can be InfoArray)
        # From new-from-template - e.g infoarr[:3]
        #    type(obj) is InfoArray
        #
        # Note that it is here, rather than in the __new__ method,
        # that we set the default value for 'info', because this
        # method sees all creation of default objects - with the
        # InfoArray.__new__ constructor, but also with
        # arr.view(InfoArray).
        self.info = getattr(obj, 'info', None)
        # We do not need to return anything

In [29]:
obj = InfoArray(shape=(3,)) # explicit constructor
type(obj)

__main__.InfoArray

In [30]:
obj.info is None

True

In [31]:
obj = InfoArray(shape=(3,), info='information')
obj.info

'information'

In [32]:
v = obj[1:] # new-from-template - here - slicing
print(type(v))
print(v.info)

<class '__main__.InfoArray'>
information


In [33]:
arr = np.arange(10)
cast_arr = arr.view(InfoArray) # view casting
print(type(cast_arr))

<class '__main__.InfoArray'>


# Internal organization of NumPy arrays

NumPy arrays consist of two major components: the raw array data (from now on, referred to as the **data buffer**), and the information about the **raw array data**. This extra information contains (among other things):

1. The basic data element’s size in bytes.
2. The start of the data within the data buffer (an offset relative to the beginning of the data buffer).
3. The number of dimensions and the size of each dimension.
4. The separation between elements for each dimension (the stride). This does not have to be a multiple of the element size.
5. The byte order of the data (which may not be the native byte order).
6. Whether the buffer is read-only.
7. Information (via the dtype object) about the interpretation of the basic data element. The basic data element may be as simple as an int or a float, or it may be a compound object (e.g., struct-like), a fixed character field, or Python object pointers.

This arrangement allows for the very flexible use of arrays. 

# Memory

The actual data of a numpy array is stored in a homogeneous and contiguous block of memory called data buffer. 

![Memory](Images/PythonMemory.png)

## Strides
Stride is a tuple of bytes to step in each dimension when traversing an array.

In [34]:
y = np.reshape(np.arange(2*3*4), (2,3,4))
y

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [35]:
y.strides

(96, 32, 8)

In [36]:
z = y.T
z.strides

(8, 32, 96)

In [37]:
y[1,1,1]

np.int64(17)

At this step, we want to calculate how many bytes we need to "jump" in memory to access the element.

In [38]:
offset=sum(y.strides * np.array((1,1,1)))
offset/y.itemsize

np.float64(17.0)

What does `y.strides * np.array((1, 1, 1))` mean?

Each dimension of the array (2, 3, 4) has its own stride, which tells us how far apart elements are in memory:
* For the first dimension (axis 0), we need to jump 96 bytes to move to the next element.
* For the second dimension (axis 1), we jump 32 bytes.
* For the third dimension (axis 2), we jump 8 bytes.

By summing these values we obtain the offset.

What about `offset / y.itemsize`?

Each element occupies 4 bytes (y.itemsize == 4). To find the position of the element in the flattened 1D representation, we divide `offset / y.itemsize`.

## Changing shape

The shape of the array can be changed very easily without changing anything in the data buffer or any data copying at all. To reshape the array in-place assume a tuple of array dimensions to it. Reshaping an array in-place will fail if a copy is required.

In [39]:
x = np.array([1, 2, 3, 4])
x.shape

(4,)

In [40]:
y = np.zeros((2, 3, 4))
y.shape

(2, 3, 4)

In [41]:
y.shape = (3, 8)
y

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [42]:
# # ValueError: total size of new array must be unchanged
# y.shape = (3, 6)

In [43]:
# # AttributeError: Incompatible shape for in-place modification. Use `.reshape()` to make a copy with the desired shape.
# np.zeros((4,2))[::2].shape = (-1,)

## Numpy data structures

### numpy.lib.stride_tricks.as_strided

The `numpy.lib.stride_tricks.as_strided()` function allows you to create custom views of NumPy arrays without copying data. It works by manipulating memory strides, enabling you to define a custom way of interpreting the underlying data.

**Function Parameters**

| Parameter     | Description                                           |
|---------------|-------------------------------------------------------|
| `x`           | Input NumPy array                                     |
| `shape`       | Desired shape of the new view                         |
| `strides`     | Bytes to move between elements in each dimension      |
| `subok`       | Keeps subclass type if `True` (default)               |
| `writeable`   | If `True`, allows changes to the view (use cautiously)|

**Returns**

`view : ndarray`
A new view of the input array with overlapping sliding windows.


**Risks**
1. **Memory violation risk** Manipulating strides allows reading data outside the valid array range, which can cause errors or undefined behavior.
2. **No data copying** Modifying elements of a view created with as_strided will also modify the original data.

#### Example
It can be used to efficiently perform operations on input data, such as creating sliding windows without memory overhead. This is particularly useful for convolutional operations.

In [44]:
from numpy.lib import stride_tricks

x = np.arange(10)

window_size = 3
stride = x.strides[0]

windows = stride_tricks.as_strided(x, 
                                   shape=(len(x) - window_size + 1, window_size), 
                                   strides=(stride, stride))
print(windows)

[[0 1 2]
 [1 2 3]
 [2 3 4]
 [3 4 5]
 [4 5 6]
 [5 6 7]
 [6 7 8]
 [7 8 9]]


**How It Works**

1. Create the input array x
2. Define the window size and stride
    * window_size = 3: We want overlapping sliding windows of length 3.
    * x.strides[0]: The stride for a 1D NumPy array is the number of bytes required to move from one element to the next.
3. Set the shape of the new view `shape=(len(x) - window_size + 1, window_size)`
    * len(x) = 10
    * We want windows of size 3, so the number of possible windows is 10 - 3 + 1 = 8. This makes the shape of the resulting array (8, 3).
4. Set the strides `strides=(stride, stride)`
    * The first value tells NumPy how many bytes to move when selecting the starting point of the next window (row).
    * The second value tells NumPy how many bytes to move between elements within a window (column).


### numpy.lib.stride_tricks.sliding_window_view

The `numpy.lib.stride_tricks.sliding_window_view()` function creates sliding window views of N-dimensional arrays without copying data. This function is safer and easier to use than `as_strided()` for tasks like generating moving windows for convolution or analysis.

**Function Parameters**

| Parameter        | Description                                                  |
|------------------|--------------------------------------------------------------|
| `x`              | Input NumPy array                                            |
| `window_shape`   | Size of the sliding window (int or tuple for N-dim arrays)   |
| `axis` (optional) | Axis or axes to slide over; defaults to all axes            |

**Returns**

`view : ndarray`  
A new view of the input array with overlapping sliding windows.


#### Example

In [45]:
x = np.arange(10)

windows = stride_tricks.sliding_window_view(x, window_shape=3)
print(windows)

[[0 1 2]
 [1 2 3]
 [2 3 4]
 [3 4 5]
 [4 5 6]
 [5 6 7]
 [6 7 8]
 [7 8 9]]


# Assignments

1. Strides, Views
   
   Generate a 5×4 array A containing consecutive natural numbers.
   Create array B as a view of A.
   Modify the strides of B so that B becomes the transpose of A (B = A.T). Use B.strides = ? to achieve this.
   Calculate how many bytes need to be traversed in memory to access the element with the value 7 in both arrays.
2. Broadcasting

   Generate 10,000 points in a 1,000-dimensional space.
   Using broadcasting, compute the matrix of Euclidean distances between all pairs of points.
   Compare the execution time of this approach with a triple for-loop implementation.
3. Copies and Views

   Generate a 3×3 array A containing integers from 1 to 9.
   Perform the following experiments and answer the questions:
   - Create array B as a copy of A using np.copy(), and array C as a view of A using np.view().
   Modify A[0,0] to 100 in the original array A. Then modify C[2,2] to 200 in C.
   Observe and explain the changes in B, C, and A. Why did some arrays change while others didn’t?
   - Create a view D of A with an added dimension (for example, using np.newaxis in the appropriate position) and observe how the data changes.
   Modify a row in C (for example, C[1,:]) and check if the changes affect A. Then, check whether modifying B affects A.
4. Subclassing

   Implement a custom subclass MyArray, inheriting from np.ndarray.
   Add a method norm() that computes the matrix norm using a selected norm type.
   Verify the correctness of MyArray and the norm() method for different values.
5. Stride Tricks

   Create an array as shown on the left side of the diagram. Using stride_tricks, transform it into the array on the right side of the diagram by swapping the top-right quadrant with the bottom-left.
   ![task](Images/ex_np_as_strided.png)

## 1. Strides, Views
   
   Generate a 5×4 array A containing consecutive natural numbers.
   Create array B as a view of A.
   Modify the strides of B so that B becomes the transpose of A (B = A.T). Use B.strides = ? to achieve this.
   Calculate how many bytes need to be traversed in memory to access the element with the value 7 in both arrays.

In [46]:
A = np.arange(1, 21).reshape(5, 4)
B = A.view()
B.strides = (A.strides[1], A.strides[0])

# Memory traversal for element 7
index_A = np.where(A == 7)
memory_offset_A = index_A[0][0] * A.strides[0] + index_A[1][0] * A.strides[1]
index_B = np.where(B == 7)
memory_offset_B = index_B[0][0] * B.strides[0] + index_B[1][0] * B.strides[1]

memory_offset_A, memory_offset_B

(np.int64(48), np.int64(48))

2. Broadcasting

   Generate 10,000 points in a 1,000-dimensional space.
   Using broadcasting, compute the matrix of Euclidean distances between all pairs of points.
   Compare the execution time of this approach with a triple for-loop implementation.

In [47]:
num_points = 10_000
dimensions = 1_000
X = np.random.randn(num_points, dimensions).astype(np.float16)

def broadcasting(X):
	X_norm = np.sum(X**2, axis=1)
	return X_norm[:, np.newaxis] + X_norm[np.newaxis, :] - 2 * np.dot(X, X.T)

def triple_loop(X):
    num_points = X.shape[0]
    dist_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(num_points):
            dist_matrix[i, j] = np.sqrt(np.sum((X[i] - X[j]) ** 2))
    return dist_matrix

In [49]:
%timeit broadcasting(X)

5min 14s ± 3.71 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 3. Copies and Views

   Generate a 3×3 array A containing integers from 1 to 9.
   Perform the following experiments and answer the questions:
   - Create array B as a copy of A using np.copy(), and array C as a view of A using np.view().
   Modify A[0,0] to 100 in the original array A. Then modify C[2,2] to 200 in C.
   Observe and explain the changes in B, C, and A. Why did some arrays change while others didn’t?
   - Create a view D of A with an added dimension (for example, using np.newaxis in the appropriate position) and observe how the data changes.
   Modify a row in C (for example, C[1,:]) and check if the changes affect A. Then, check whether modifying B affects A.

In [ ]:
A = np.arange(1, 10).reshape(3, 3)
B = np.copy(A)
C = A.view()
A[0, 0] = 100
C[2, 2] = 200

In [ ]:
A

array([[100,   2,   3],
       [  4,   5,   6],
       [  7,   8, 200]])

In [ ]:
B

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [ ]:
C

array([[100,   2,   3],
       [  4,   5,   6],
       [  7,   8, 200]])

C changed since it doesn't own the values and merely keeps pointers to them unlike B which is completely separate in memory.

In [ ]:
D = A[np.newaxis, :, :]
C[1,:]=1

In [ ]:
A

array([[100,   2,   3],
       [  1,   1,   1],
       [  7,   8, 200]])

In [ ]:
D

array([[[100,   2,   3],
        [  1,   1,   1],
        [  7,   8, 200]]])

## 4. Subclassing

   Implement a custom subclass MyArray, inheriting from np.ndarray.
   Add a method norm() that computes the matrix norm using a selected norm type.
   Verify the correctness of MyArray and the norm() method for different values.

In [ ]:
class MyArray(np.ndarray):
    def norm(self, ord=None):
        return np.linalg.norm(self, ord=ord)

def create_myarray(data):
    obj = np.asarray(data).view(MyArray)
    return obj

my_arr = create_myarray(np.array([[3, 4], [1, 2]]))
norm_val = my_arr.norm()

## 5. Stride Tricks

   Create an array as shown on the left side of the diagram. Using stride_tricks, transform it into the array on the right side of the diagram by swapping the top-right quadrant with the bottom-left.

In [ ]:
original = np.array([[1, 2, 3, 4],
                     [5, 6, 7, 8],
                     [9, 10, 11, 12],
                     [13, 14, 15, 16]])


top_left = original[:2, :2]
top_right = original[:2, 2:]
bottom_left = original[2:, :2]
bottom_right = original[2:, 2:]

swapped = np.block([[top_left, bottom_left],
                    [top_right, bottom_right]])
swapped

array([[ 1,  2,  9, 10],
       [ 5,  6, 13, 14],
       [ 3,  4, 11, 12],
       [ 7,  8, 15, 16]])

# Bibliography

[Internal organization of NumPy arrays](https://numpy.org/doc/stable/dev/internals.html)

[Guide to NumPy](https://web.mit.edu/dvp/Public/numpybook.pdf)

[NumPy MedKit](https://mentat.za.net/numpy/numpy_advanced_slides/)

[Python Fundamentals](https://numpy.org/doc/2.2/user/basics.html)

[Copy and View](https://www.w3schools.com/python/numpy/numpy_copy_vs_view.asp)

[Subclassing ndarray](https://numpy.org/doc/stable/user/basics.subclassing.html)

[Understanding the Difference Between Shallow Copy, Deep Copy, and References in Python](https://www.linkedin.com/pulse/understanding-difference-between-shallow-copy-deep-python-torabi-nnh5e/)